In my windows Anaconda installation the %matplotlib tk didn't work, the qt4 worked
This is bit funny I am not sure about this part yet. I', leaving this for future investigarion 

In [1]:

%matplotlib qt5
#%matplotlib tk 
#tk did not work for me in windows anaconda

import pyaudio
import struct
import numpy as np 
import matplotlib.pyplot as plt

import os
import time
from scipy.fftpack import fft

CHUNK = 1024*2
FORMAT = pyaudio.paInt16
CHANNELS =1 
RATE =44100


In [6]:
fig,(ax1, ax2) = plt.subplots(2, figsize=(15,7))

p= pyaudio.PyAudio()
stream = p.open(
    format= FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)
#data = stream.read(CHUNK)
#data_int =np.array(struct.unpack(str(2*CHUNK)+'B',data),dtype='b')[::2]+127

#variable for plotting 
x =np.arange(0,2*CHUNK,2)   #samples (waveform)
xf = np.linspace(0, RATE, CHUNK) #frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-',lw=2) # put a random number for now

# create semixlogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')

ax1.set_ylim(-32768 ,32768 ) #t
ax1.set_xlim(0,CHUNK)

#format spectrum axes
ax2.set_xlim(20, RATE/2)



(20, 22050.0)

we want to update the line data in real time 

In [ ]:
#for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    data = stream.read(CHUNK)
    #data_int = np.array(struct.unpack(str(2*CHUNK)+'B', data), dtype='b')[::2] + 127
    #data_int = np.fromstring(data,dtype=np.int16)
    data_int = np.frombuffer(stream.read(CHUNK),np.int16,-1)
    line.set_ydata(data_int)
    
    #compute FFT and update line
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK]) / (128*CHUNK))
    
    fig.canvas.draw()
    fig.canvas.flush_events()
    frame_count += 1